In [2]:
import os
import math
import numpy as np
import xarray as xr
from joblib import Parallel, delayed

def area(dir_path, resolution_deg, region=[-90,90,-180,180], region_name='', south2north=False):
    """
    Calculate grid cell areas for a specified lat/lon range or global grid.
    """

    lat_min, lat_max, lon_min, lon_max = region
    # Directory setup
    os.makedirs(dir_path, exist_ok=True)
    
    # Set output file name based on range
    if float(resolution_deg).is_integer():
        deg_name = str(resolution_deg)+'p'
    else:
        deg_name = str(resolution_deg).replace('.', 'p')

    if lat_min==-90 and lat_max==90 and lon_min==-180 and lon_max==180:
        area_file = os.path.join(dir_path, f'Area_{deg_name}.nc')
    else:
        area_file = os.path.join(dir_path, f'Area_{deg_name}_{region_name}.nc')

    def count_area(lat1, lat2, dlon_rad):
        """Calculate area of a grid cell given latitude bounds and longitude width."""
        lat1, lat2 = map(math.radians, [lat1, lat2])
        R = 6.37122e6  # Earth's radius in meters
        area = abs(R**2 * dlon_rad * (math.sin(lat2) - math.sin(lat1)))
        return area

    # Validate resolution
    if resolution_deg <= 0:
        raise ValueError("Resolution must be positive")

    # Set default global grid if no range is specified
    lat_min = -90 if lat_min is None else lat_min
    lat_max = 90 if lat_max is None else lat_max
    lon_min = -180 if lon_min is None else lon_min
    lon_max = 180 if lon_max is None else lon_max

    # Validate latitude and longitude ranges
    if not (-90 <= lat_min < lat_max <= 90):
        raise ValueError("Latitude range must satisfy -90 <= lat_min < lat_max <= 90")
    if not (-180 <= lon_min < lon_max <= 180):
        raise ValueError("Longitude range must satisfy -180 <= lon_min < lon_max <= 180")

    # Create lat/lon grid based on resolution and specified range
    inc = resolution_deg  # Grid spacing in degrees
    if south2north:
        lat = np.arange(lat_min + inc/2, lat_max + inc/2, inc)  # Center of grid cells
    else:
        lat = np.arange(lat_max - inc/2, lat_min - inc/2, -inc) 
    print(lat)
    lon = np.arange(lon_min + inc/2, lon_max + inc/2, inc)
    dlon_rad = math.radians(inc)  # Longitude width in radians

    # Ensure grid is not empty
    if len(lat) == 0 or len(lon) == 0:
        raise ValueError("Specified range is too small for the given resolution")

    # Create meshgrid for area calculation
    grid1, grid2 = np.meshgrid(lon, lat)
    area = np.zeros_like(grid1)

    # Calculate area for each latitude band
    lat1 = lat - inc/2  # Lower bound of each grid cell
    lat2 = lat + inc/2  # Upper bound of each grid cell
    result = Parallel(n_jobs=12)(delayed(count_area)(lat1[i], lat2[i], dlon_rad) for i in range(len(lat)))

    # Assign area values to the grid
    for i in range(len(lat)):
        area[i, :] = result[i]
        print(f"Area at lat {lat[i]:.2f}°: {area[i,0]:.3f} m²")

    # Print total area
    total_area_m2 = np.sum(area)
    print(f'The total area: {total_area_m2/1e12:.3f} million km²')

    # Create xarray Dataset
    output_ds = xr.Dataset(
        {'area': (('lat', 'lon'), area)},
        coords={'lat': lat, 'lon': lon}
    )
    output_ds.attrs['units'] = 'm2'
    output_ds['lat'].attrs['units'] = 'degrees_north'
    output_ds['lon'].attrs['units'] = 'degrees_east'
    print(output_ds)
    # Save to NetCDF
    output_ds.to_netcdf(area_file)
    print(f"Area calculation completed, saved to {area_file}")

In [3]:
output_path = '/share/home/dq076/data/Area'
resolution = 0.5
# region = [-90,90,-180,180]
# south2north=False
area(output_path, resolution)

[ 89.75  89.25  88.75  88.25  87.75  87.25  86.75  86.25  85.75  85.25
  84.75  84.25  83.75  83.25  82.75  82.25  81.75  81.25  80.75  80.25
  79.75  79.25  78.75  78.25  77.75  77.25  76.75  76.25  75.75  75.25
  74.75  74.25  73.75  73.25  72.75  72.25  71.75  71.25  70.75  70.25
  69.75  69.25  68.75  68.25  67.75  67.25  66.75  66.25  65.75  65.25
  64.75  64.25  63.75  63.25  62.75  62.25  61.75  61.25  60.75  60.25
  59.75  59.25  58.75  58.25  57.75  57.25  56.75  56.25  55.75  55.25
  54.75  54.25  53.75  53.25  52.75  52.25  51.75  51.25  50.75  50.25
  49.75  49.25  48.75  48.25  47.75  47.25  46.75  46.25  45.75  45.25
  44.75  44.25  43.75  43.25  42.75  42.25  41.75  41.25  40.75  40.25
  39.75  39.25  38.75  38.25  37.75  37.25  36.75  36.25  35.75  35.25
  34.75  34.25  33.75  33.25  32.75  32.25  31.75  31.25  30.75  30.25
  29.75  29.25  28.75  28.25  27.75  27.25  26.75  26.25  25.75  25.25
  24.75  24.25  23.75  23.25  22.75  22.25  21.75  21.25  20.75  20.25
  19.7

Area at lat 89.75°: 13488217.714 m²
Area at lat 89.25°: 40463625.961 m²
Area at lat 88.75°: 67435952.747 m²
Area at lat 88.25°: 94403144.024 m²
Area at lat 87.75°: 121363146.135 m²
Area at lat 87.25°: 148313905.974 m²
Area at lat 86.75°: 175253371.134 m²
Area at lat 86.25°: 202179490.072 m²
Area at lat 85.75°: 229090212.258 m²
Area at lat 85.25°: 255983488.338 m²
Area at lat 84.75°: 282857270.284 m²
Area at lat 84.25°: 309709511.554 m²
Area at lat 83.75°: 336538167.246 m²
Area at lat 83.25°: 363341194.253 m²
Area at lat 82.75°: 390116551.422 m²
Area at lat 82.25°: 416862199.704 m²
Area at lat 81.75°: 443576102.317 m²
Area at lat 81.25°: 470256224.891 m²
Area at lat 80.75°: 496900535.634 m²
Area at lat 80.25°: 523507005.477 m²
Area at lat 79.75°: 550073608.234 m²
Area at lat 79.25°: 576598320.757 m²
Area at lat 78.75°: 603079123.085 m²
Area at lat 78.25°: 629513998.603 m²
Area at lat 77.75°: 655900934.193 m²
Area at lat 77.25°: 682237920.387 m²
Area at lat 76.75°: 708522951.522 m²
Area 

In [19]:
output_path = '/share/home/dq076/data/Area'
resolution = 0.5
region = [-17,5,-80,-50]
region_name = 'Amazonia'
# south2north=False
area(output_path, resolution, region, region_name)

[  4.75   4.25   3.75   3.25   2.75   2.25   1.75   1.25   0.75   0.25
  -0.25  -0.75  -1.25  -1.75  -2.25  -2.75  -3.25  -3.75  -4.25  -4.75
  -5.25  -5.75  -6.25  -6.75  -7.25  -7.75  -8.25  -8.75  -9.25  -9.75
 -10.25 -10.75 -11.25 -11.75 -12.25 -12.75 -13.25 -13.75 -14.25 -14.75
 -15.25 -15.75 -16.25 -16.75]
Area at lat 4.75°: 3080664602.591 m²
Area at lat 4.25°: 3082781149.321 m²
Area at lat 3.75°: 3084662930.330 m²
Area at lat 3.25°: 3086309802.315 m²
Area at lat 2.75°: 3087721639.859 m²
Area at lat 2.25°: 3088898335.446 m²
Area at lat 1.75°: 3089839799.465 m²
Area at lat 1.25°: 3090545960.221 m²
Area at lat 0.75°: 3091016763.936 m²
Area at lat 0.25°: 3091252174.758 m²
Area at lat -0.25°: 3091252174.758 m²
Area at lat -0.75°: 3091016763.936 m²
Area at lat -1.25°: 3090545960.221 m²
Area at lat -1.75°: 3089839799.465 m²
Area at lat -2.25°: 3088898335.446 m²
Area at lat -2.75°: 3087721639.859 m²
Area at lat -3.25°: 3086309802.315 m²
Area at lat -3.75°: 3084662930.330 m²
Area at lat 